In [1]:
import pandas as pd

### Define Functions

In [2]:
# Define a function to keep the first and third occurrence of each index item
def keep_first_and_third(group):
    # Keep the first and third rows of each group
    return group.iloc[[0, 2]] if len(group) >= 3 else group

In [3]:
# Define a function to assign 'Milk' and 'StM'
def assign_region(group):
    # Create a new column 'region' with default value 'StM'
    group['region'] = 'StM'
    # Set 'Milk' for the first appearance
    group.iloc[0, group.columns.get_loc('region')] = 'Milk'
    return group

In [4]:
def write_par_table(par: str,
                    reg_par: str,
                    reg_par_key: str,
                    cal_params: str,
                    lu_classes: str,
                    st_classes: str) -> pd.DataFrame:
    
    """ This function writes a table contaning all HYPE parameters based on the outputs:
    
    par: Path to the par.txt file in HYPE
    reg_par: Path to the reg_par.txt file in HYPE
    reg_par_key: Path to the key containing regionalized parameters and their regression values
    cal_params: Path to the dataframe of parameters being calibrated
    lu_classes: Path to the file containing the land use classes and what they represent
    st_classes: Path to the file containing the soil type classes and what they represent """


    # Read the file into a DataFrame, ignoring lines starting with '#'
    par = pd.read_csv(par, comment='!', delimiter='\t', header= None, index_col=0)

    # read regional parameters
    reg_par= pd.read_csv(reg_par, delim_whitespace=True, skiprows=1, header= None, index_col= 0)

    # read regional parameter regression key
    reg_par_key= pd.read_excel(reg_par_key, index_col=0)

    # read cal params
    cal_params= pd.read_csv(cal_params, delimiter='\t',header= None,index_col=0)

    # read land use classes
    lu_classes= pd.read_excel(lu_classes, sheet_name='Land Use',index_col=0)

    # read soil type classes
    st_classes= pd.read_excel(st_classes, sheet_name='Soil Type',index_col=0)
    
    # Format regionalized parameters
    
    # Apply the function to each group
    filtered_reg_par = reg_par.groupby(level=0).apply(keep_first_and_third).reset_index(level=0, drop=True)

    # Apply the function to each group
    filtered_reg_par = filtered_reg_par.groupby(0, group_keys=False).apply(assign_region)

    # Merge the DataFrames on the index
    merged_reg_par = filtered_reg_par.join(reg_par_key)

    # Multiply the values in the first column by the 'Regression' values
    merged_reg_par['Value'] = merged_reg_par[1] * merged_reg_par['Regression']

    # Drop the 'Regression' column if not needed
    merged_reg_par = merged_reg_par.drop(columns=['Regression', 1])

    # Pivot the DataFrame
    pivot_reg_par = merged_reg_par.reset_index().pivot(index='index', columns='region', values='Value')

    # Rename index to 'Parameter'
    pivot_reg_par.index.name = 'Parameter'

    # Format parameter file
    
    # Convert index of cal_params to string
    cal_params.index = cal_params.index.astype(str)

    # Change column names in cal_params to integers
    cal_params.columns = range(cal_params.shape[1])

    # Convert index of par to string
    par.index = par.index.astype(str)

    # Convert column headers of 'par' to integers
    par.columns = par.columns.astype(int)

    # Convert index of lu_classes to int
    lu_classes.index = lu_classes.index.astype(int)

    # Convert index of st_classes to int
    st_classes.index = st_classes.index.astype(int)
    
    # Create a land use mapping dictionary
    lu_mapping = dict(zip(lu_classes.index, lu_classes.iloc[:, 0]))

    # Create a soil type mapping dictionary
    st_mapping = dict(zip(st_classes.index, st_classes.iloc[:, 0]))
    
    lu_indices = par.index.intersection(cal_params.index)
    # Filter rows where the 6th column of 'cal_params' has the value 'Land Use'
    land_use = par.loc[
        lu_indices[cal_params.loc[lu_indices].iloc[:, 6] == 'Land Use']
    ]

    # Update the DataFrame's columns using the mapping dictionary
    land_use.columns = [lu_mapping.get(col, col) for col in land_use.columns]

    # Set the name of the index
    land_use = land_use.rename_axis('Parameter')
    
    st_indices = par.index.intersection(cal_params.index)
    
    # Filter rows where the 6th column of 'cal_params' has the value 'Land Use'
    soil_type = par.loc[
        st_indices[cal_params.loc[st_indices].iloc[:, 6] == 'Soil Type']
    ]

    # Update the DataFrame's columns using the mapping dictionary
    soil_type.columns = [st_mapping.get(col, col) for col in soil_type.columns]

    # Set the name of the index
    soil_type = soil_type.rename_axis('Parameter')

    # Drop any column that contains NaN values
    soil_type = soil_type.dropna(axis=1)
    
    general_indices = par.index.intersection(cal_params.index)
    # Filter rows where the 6th column of 'cal_params' has the value 'Land Use'
    general = par.loc[
        general_indices[cal_params.loc[general_indices].iloc[:, 6] == 'General']
    ]

    # Set the name of the index
    general = general.rename_axis('Parameter')

    # Drop any column that contains NaN values
    general = general.dropna(axis=1)

    general.rename(columns={general.columns[0]: 'General'}, inplace=True)
    
    # Concatenate the DataFrames with different headers and indexes
    combined_par = pd.concat([land_use, soil_type, general, pivot_reg_par], keys=['Land Use', 'Soil Type', 'General', 'Regionalized General'])

    # Round values to 3 significant figures
    combined_par = combined_par.round(3)
    
    return combined_par

In [5]:
def filter_by_sensitivity(par: pd.DataFrame)-> pd.DataFrame:
    
    # List of keywords to filter the columns (most dominant lu/st)
    keywords = ['Temperate or sub-polar shrubland', 
                'Temperate or sub-polar grassland', 
                'Cropland', 
                '_Clay'] # this will keep only Clay and Clay loam, otherwise it keeps anything with clay in it
    
    # Creating a boolean mask for column selection based on keywords
    mask = par.columns.str.contains('|'.join(keywords), case=False)

    # Filtering the DataFrame to keep only the desired columns
    filtered_par= par.loc[:, mask]
    
    return filtered_par

In [6]:
def find_difference(par1: pd.DataFrame, par2: pd.DataFrame)-> pd.DataFrame:
    
    # Check if both DataFrames have the same index and columns
    if par1.index.equals(par2.index) and par1.columns.equals(par2.columns):
        # Subtract par2 from par1
        result = par1 - par2
        
        return result
    
    else:
        mismatch_info = []
        if not par1.index.equals(par2.index):
            mismatch_info.append(f"Indexes do not match: par1 index = {par1.index}, par2 index = {par2.index}")
        if not par1.columns.equals(par2.columns):
            mismatch_info.append(f"Headers do not match: par1 headers = {par1.columns}, par2 headers = {par2.columns}")
            
        return "\n".join(mismatch_info)

### Inputs

In [7]:
# Define paths that apply to all parameter sets
# read regional parameter regression key
reg_par_key_path= '../../geospacial/misc/reg_par_key.xlsx'

# read cal params
cal_params_path= '../../calibration/cal_param.txt'

# read land use classes
lu_classes_path= '../../geospacial/misc/lu_st_classes.xlsx'

# read soil type classes
st_classes_path= '../../geospacial/misc/lu_st_classes.xlsx'

### Analysis

In [8]:
# Define model specific parameter paths
par_path_nse= '../../model/model_versions/v_7/v7_1/par.txt'

# read regional parameters
reg_par_path_nse= '../../model/model_versions/v_7/v7_1/reg_par.txt'

# generate table
par_nse= write_par_table(par_path_nse, reg_par_path_nse, reg_par_key_path, 
                         cal_params_path, lu_classes_path, st_classes_path)

In [9]:
# # Define model specific parameter paths
# par_path_kge= '../../model/seperate_cal/01_kge_combined_model/par.txt'

# # read regional parameters
# reg_par_path_kge= '../../model/seperate_cal/01_kge_combined_model/reg_par.txt'

# # generate table
# par_kge= write_par_table(par_path_kge, reg_par_path_kge, reg_par_key_path, 
#                          cal_params_path, lu_classes_path, st_classes_path)

In [10]:
# filter by most dominant lu and st
par_nse_filtered= filter_by_sensitivity(par_nse)
# par_kge_filtered= filter_by_sensitivity(par_kge)

In [11]:
# find difference (par1-par2)
# difference= find_difference(par_nse_filtered, par_kge_filtered)

In [12]:
par_nse_filtered

04_Milk_Temperate or sub-polar shrubland  \
                     Parameter                                              
Land Use             ttmp                                          -1.340   
                     cmlt                                           4.089   
                     kc3                                            0.776   
                     alb                                            0.262   
                     srrcs                                          0.013   
Soil Type            bfroznsoil                                       NaN   
                     bcosby                                           NaN   
                     rrcs1                                            NaN   
                     rrcs2                                            NaN   
                     srrate                                           NaN   
                     wcwp1                                            NaN   
                     wcwp2                                            NaN   
                     wcwp3                                            NaN   
                     wcfc1                                            NaN   
                     wcfc2                                            NaN   
                     wcfc3                                            NaN   
                     wcep1                                            NaN   
                     wcep2                                            NaN   
                     wcep3                                            NaN   
                     mperc1                                           NaN   
                     mperc2                                           NaN   
                     macrate                                          NaN   
                     mactrinf                                         NaN   
                     mactrsm                                          NaN   
General              wetrate                                          NaN   
                     wetexp                                           NaN   
                     iwetw0                                           NaN   
Regionalized General damp                                             NaN   
                     lp                                               NaN   
                     rivvel                                           NaN   
                     rrcs3                                            NaN   

                                 05_Milk_Temperate or sub-polar grassland  \
                     Parameter                                              
Land Use             ttmp                                          -1.171   
                     cmlt                                           1.074   
                     kc3                                            1.160   
                     alb                                            0.220   
                     srrcs                                          0.138   
Soil Type            bfroznsoil                                       NaN   
                     bcosby                                           NaN   
                     rrcs1                                            NaN   
                     rrcs2                                            NaN   
                     srrate                                           NaN   
                     wcwp1                                            NaN   
                     wcwp2                                            NaN   
                     wcwp3                                            NaN   
                     wcfc1                                            NaN   
                     wcfc2                                            NaN   
                     wcfc3                                            NaN   
                     wcep1                                            NaN   
                     wcep2  

In [13]:
par_nse_filtered

04_Milk_Temperate or sub-polar shrubland  \
                     Parameter                                              
Land Use             ttmp                                          -1.340   
                     cmlt                                           4.089   
                     kc3                                            0.776   
                     alb                                            0.262   
                     srrcs                                          0.013   
Soil Type            bfroznsoil                                       NaN   
                     bcosby                                           NaN   
                     rrcs1                                            NaN   
                     rrcs2                                            NaN   
                     srrate                                           NaN   
                     wcwp1                                            NaN   
                     wcwp2                                            NaN   
                     wcwp3                                            NaN   
                     wcfc1                                            NaN   
                     wcfc2                                            NaN   
                     wcfc3                                            NaN   
                     wcep1                                            NaN   
                     wcep2                                            NaN   
                     wcep3                                            NaN   
                     mperc1                                           NaN   
                     mperc2                                           NaN   
                     macrate                                          NaN   
                     mactrinf                                         NaN   
                     mactrsm                                          NaN   
General              wetrate                                          NaN   
                     wetexp                                           NaN   
                     iwetw0                                           NaN   
Regionalized General damp                                             NaN   
                     lp                                               NaN   
                     rivvel                                           NaN   
                     rrcs3                                            NaN   

                                 05_Milk_Temperate or sub-polar grassland  \
                     Parameter                                              
Land Use             ttmp                                          -1.171   
                     cmlt                                           1.074   
                     kc3                                            1.160   
                     alb                                            0.220   
                     srrcs                                          0.138   
Soil Type            bfroznsoil                                       NaN   
                     bcosby                                           NaN   
                     rrcs1                                            NaN   
                     rrcs2                                            NaN   
                     srrate                                           NaN   
                     wcwp1                                            NaN   
                     wcwp2                                            NaN   
                     wcwp3                                            NaN   
                     wcfc1                                            NaN   
                     wcfc2                                            NaN   
                     wcfc3                                            NaN   
                     wcep1                                            NaN   
                     wcep2  

In [14]:
difference

NameError: name 'difference' is not defined